In [ ]:
###################Encryption, Update and Keystream function definition Start#####################

def keystream(L,N):                 #Keystream bit function
    z = L[30] ^ N[1] ^ N[6] ^ N[15] ^  N[17] ^ N[23] ^ N[28] ^ N[34] ^ (N[4]&L[6] ^ L[8]&L[10] ^ L[32]&L[17] ^ L[19]&L[23] ^ N[4]&L[32]&N[38])
    return z
    
def LFSR_fw_update(L):              #LFSR feedback function in forward direction
    l = (L[0] ^ L[14] ^ L[20] ^ L[34] ^ L[43] ^ L[54])
    return l
    
def LFSR_bk_update(L):              #LFSR feedback function in backward direction
    lb = L[60] ^ L[13] ^ L[19] ^ L[33] ^ L[42] ^ L[53]
    return lb

def NFSR_fw_update(N,l0,key,t):     #NFSR feedback function in forward direction
    n = l0 ^ (((t%80)>>4)&1) ^ key[t%80] ^ N[0] ^ N[13] ^ N[19] ^ N[35] ^ N[39] ^ N[2]&N[25] ^ N[3]&N[5] ^ N[7]&N[8] ^ N[14]&N[21] ^ N[16]&N[18] ^ N[22]&N[24] ^ N[26]&N[32] ^ N[33]&N[36]&N[37]&N[38] ^ N[10]&N[11]&N[12] ^ N[27]&N[30]&N[31]
    return n
    
def NFSR_bk_update(N,l0,key,t):     #NFSR feedback function in backward direction
    nb = l0 ^ (((t%80)>>4)&1) ^ key[t%80] ^ N[39] ^ N[12] ^ N[18] ^ N[34] ^ N[38] ^ N[1]&N[24] ^ N[2]&N[4] ^ N[6]&N[7] ^ N[13]&N[20] ^ N[15]&N[17] ^ N[21]&N[23] ^ N[25]&N[31] ^ N[32]&N[35]&N[36]&N[37] ^ N[9]&N[10]&N[11] ^ N[26]&N[29]&N[30]
    return nb


def Encryption():                                #Encryption function
    import random
    key = [random.randint(0,1) for i in range(80)]
    IV = [random.randint(0,1) for i in range(90)]
    N = [IV[i] for i in range(40)]                                      #NFSR initialisation
    L = [IV[40+i] for i in range(50)] + [1,1,1,1,1,1,1,1,1,0,1]         #LFSR initialisation
    
    #Initialisation Phase
    for i in range(320):
        z = keystream(L,N)                         #keystream bit
        l_new = (z ^ LFSR_fw_update(L))            #LFSR feedback bit
        l0 = L[0]
        L = L[1:60] + [l_new,1]                    #LFSR update
    
        n_new = (z ^ NFSR_fw_update(N,l0,key,i))   #NFSR feedback bit
        N = N[1:40] + [n_new]                      #NFSR update
        
    
    #Pseudorandom (PRGA) phase
    key_stream = 100                          #Targeted round (any random number)
    for i in range(key_stream):

        z = keystream(L,N)

        l_new = LFSR_fw_update(L)
        l0 = L[0]
        L = L[1:61] + [l_new]

        n_new = NFSR_fw_update(N,l0,key,i)
        N = N[1:40] + [n_new]
    
    return N,L,key                                 
        
####################Encryption, Update and Keystream function definition End#####################




#Plantlet Encryption
#Specificaton
#101-bit internal state
#NFSR 40-bit  LFSR 61-bit
#NFSR N       LFSR L
#Key 80-bit
#IV  90-bit


import math
import random
import time


state_len = 101      #state_size
mc_len = 32         #microcontroller length
N_round = 81           #Total number of rounds needed
r = [61, 40]         #Size of the registers LFSR (L) and NFSR (N) respectively

Nblock = math.ceil(state_len/mc_len)      #No. of Blocks


bitlen = math.floor(math.log(mc_len,2)) + 1   #Number of bit that is needed to represent the max. HW of any blocks
           


N_round_f = math.ceil(N_round/2)                #No. of forward rounds
N_round_b = math.floor(N_round/2)               #No. of backward rounds

print("HW/32 Model Plantlet")
print("Number of rounds in forward ", N_round_f," and in backward ", N_round_b)
print("Total number of rounds",N_round)

###############Plantlet cipher with a random key and IV to generate keystream and HW ####################################

N_org, L_org, K_org = Encryption()         #Internal state picked from the pseudorandom phase
                                           #K_org is the secret key. We need it while updating the state 
    
L = L_org[:]
N = N_org[:]

Z_f = [0]*N_round_f              #Array to store the forward keystream bits
Z_b = [0]*N_round_b              #Array to store the backward keystream bits

#Array to store the HW of each consecutive blocks in forward directions
hamm_wt_f = [[0 for i in range(Nblock)] for i in range(N_round_f)]   

#Array to store the HW of each consecutive blocks in backward directions
hamm_wt_b = [[0 for i in range(Nblock)] for i in range(N_round_b)]


#Information on keystream and HW/32 for backward direction
for i in range(N_round_b): 
    l = LFSR_bk_update(L)                    #Feedback function of LFSR
    n = NFSR_bk_update(N,l,K_org,(79-i))          #Feedback function of NFSR 
    L = [l] + L[:r[0]-1]                  #State update
    N = [n] + N[:r[1]-1] 

    temp = N + L
    
    Z_b[i] = keystream(L,N)            #Storing the keystream bits at each round in backward direction

    
    #Storing the HW of each block for internal  states in backward direction
    count = 0
    for l in range(Nblock-1):
        for k in range(mc_len):
            hamm_wt_b[i][l] += temp[count]                
            count += 1
            
    while(count < state_len):                     #Helpful when state_len is not a multiple of mc_len
        hamm_wt_b[i][-1] += temp[count]
        count += 1
    
    
            
L = L_org[:]
N = N_org[:]
#Information on keystream and HW/32 for forward direction
for i in range(N_round_f):
    Z_f[i] = keystream(L,N)            #Storing the keystream bits at each round in forward direction
    temp = N + L
    
    count = 0
    #Storing the actual HW of each block for internal  states in forward direction
    for l in range(Nblock-1):
        for k in range(mc_len):
            hamm_wt_f[i][l] += temp[count]
            count += 1
            
    while(count < state_len):                    #Helpful when state_len is not a multiple of mc_len
        hamm_wt_f[i][-1] += temp[count]
        count += 1
            
    if i < N_round_f -1:
        #updating the registers
        l = LFSR_fw_update(L)
        n = NFSR_fw_update(N,L[0],K_org,i)
        
        L = L[1:r[0]] + [l]
        N = N[1:r[1]] + [n]
        


###############SMT Modelling to recover state bits from keystream and HW/32#########################################
from z3 import *
m = Solver()

#Defining variables of the format BitVec(.,1)
L_var_org = [BitVec('l%d'%i,1) for i in range(r[0])]          #For register L
N_var_org = [BitVec('n%d'%i,1) for i in range(r[1])]          #For register N
K = [BitVec('k%d'%i,1) for i in range(len(K_org))]

L_f = [BitVec('L_f%d'%i,1) for i in range(1,N_round_f,1)]       #Dummy variables for LFSR L in forward direction
N_f = [BitVec('N_f%d'%i,1) for i in range(1,N_round_f,1)]       #Dummy variables for NFSR N in forward direction


L_b = [BitVec('L_b%d'%i,1) for i in range(1,N_round_b+1,1)]     #Dummy variables for LFSR L in backward direction
N_b = [BitVec('N_b%d'%i,1) for i in range(1,N_round_b+1,1)]     #Dummy variables for NFSR N in backward direction




L = L_var_org[:]
N = N_var_org[:]

#To check the satisfiability of equations guess all variables otherwise guess as per the requirement
#L_org, N_org are the original register of the targeted internal state
#K_org is the original secret key bits

#print("The guessed bits are ")
#print("For register L ",end=" = ")
#for i in range(r[0]):
#    print(i,end=", ")
#    m.add(L[i] == L_org[i])
#print("\n For register N ",end=" = ")
#for i in range(r[1]):
#    print(i,end=", ")
#    m.add(N[i] == N_org[i])
#print("\n For Key",end=" = ")
#for i in range(len(K_org)):
#    print(i,end=", ")
#    m.add(K[i] == K_org[i])
#print("\n")


#Array to store the keystream equations    
Z_f_equ = [0]*N_round_f        
Z_b_equ = [0]*N_round_b

#Array to store the dummy variable equations for both register L and N
L_f_equ = [0]*(N_round_f-1)
N_f_equ = [0]*(N_round_f-1)

L_b_equ = [0]*N_round_b
N_b_equ = [0]*N_round_b

#Array to store the HW/32 equations
hamm_wt_f_equ = [[0 for i in range(Nblock)] for i in range(N_round_f)] 
hamm_wt_b_equ = [[0 for i in range(Nblock)] for i in range(N_round_b)] 


#Equation generation steps for backward internal states
for i in range(N_round_b):
    
    #Dummy Variable equated to the feedback function
    l = LFSR_bk_update(L)
    n = NFSR_bk_update(N,l,K,79-i)
    
    L_b_equ[i] = (L_b[i] == l)          
    N_b_equ[i] = (N_b[i] == n)
    
    #State updated with dummy variables
    L = [L_b[i]] + L[:r[0]-1]
    N = [N_b[i]] + N[:r[1]-1]

    #Hamming weight equation
    temp = N + L
    count = 0
    for l in range(Nblock-1):            
        for k in range(mc_len):
            hamm_wt_b_equ[i][l] += ZeroExt(bitlen-1,temp[count])
            count+=1
        m.add(hamm_wt_b_equ[i][l] == hamm_wt_b[i][l])
    
    while(count < state_len):                              #Helpful when state_len is not a multiple of mc_len
        hamm_wt_b_equ[i][-1] += ZeroExt(bitlen-1,temp[count])
        count += 1
    m.add(hamm_wt_b_equ[i][-1] == hamm_wt_b[i][-1])
    
        
    #Keystream equation
    Z_b_equ[i] = (keystream(L,N) == Z_b[i])

    
L = L_var_org[:]
N = N_var_org[:]
#Equations generation steps for forward internal states
for i in range(N_round_f):
    Z_f_equ[i] = (keystream(L,N) == Z_f[i])             #Keystream equation
    
    #Hamming weight equation
    temp = N + L
    count = 0
    for l in range(Nblock-1):
        for k in range(mc_len):
            hamm_wt_f_equ[i][l] += ZeroExt(bitlen-1,temp[count])
            count+=1
        m.add(hamm_wt_f_equ[i][l] == hamm_wt_f[i][l])
        
    while(count< state_len):                            #Helpful when state_len is not a multiple of mc_len
        hamm_wt_f_equ[i][-1] += ZeroExt(bitlen-1,temp[count])
        count += 1
    m.add(hamm_wt_f_equ[i][-1] == hamm_wt_f[i][-1])
    
    if i < N_round_f-1:
        l = LFSR_fw_update(L)
        n = NFSR_fw_update(N,L[0],K,i)
        
        L_f_equ[i] = (L_f[i] == l)            #Dummy variable equated to the feedback function
        N_f_equ[i] = (N_f[i] == n)
        
        #Internal state updated with dummy variable
        L = L[1:] + [L_f[i]]
        N = N[1:] + [N_f[i]]


# All Keystream and Dummy variable equations
Equ = Z_f_equ + Z_b_equ + L_f_equ + N_f_equ + L_b_equ + N_b_equ


m.add(Equ)              #Equation feeded to the model m
start = time.time()
temp = m.check()        #Check the satisfiability of the given system of equations
end = time.time()

#If Satisfiable
if temp == sat:         
    print(temp)
    soln = m.model()           #soln contains the output
    print("SMT Time = ", end - start)         #Time taken by solver to solve
    
    #Extracting the value of the internal state variables
    out_L = [0]*(r[0])
    out_N = [0]*(r[1])
    out_K = [0]*(len(K))
    for i in range(r[0]):
        out_L[i] = soln[L_var_org[i]]
    for i in range(r[1]):
        out_N[i] = soln[N_var_org[i]]
    for i in range(len(K)):
        out_K[i] = soln[K[i]]
    
    #Check whether the original state is same as the output state
    #L_org, N_org are the original registers whereas out_L and out_N are the register from output of SMT
    #K_org is the original secret key whereas out_K is the output key bit of SMT
    if K_org == out_K:
        print("matched")
    else:
        print("############################not matched##############################################")
        print(L_org)
        print(N_org)
        print(out_L)
        print(out_N)
    
#If the solution is Unsatisfiable, check code and equations
else:
    print("###################################unsat###################################################")
    print(temp)
    print(L_org)
    print(N_org)